## Fastai pet classification in clipper cluster



## Install required packages

In [ ]:
!pip install bentoml
!pip install fastai pandas
!pip install clipper_admin

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

## Prepare Data

In [ ]:
path = untar_data(URLs.PETS); path

In [ ]:
path_anno = path/'annotations'
path_img = path/'images'
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
#bs = 64
bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

In [ ]:
np.random.seed(2)
pat = r'/([^/]+)_\d+.jpg$'
data = ImageDataBunch.from_name_re(
    path_img, fnames, pat, ds_tfms=get_transforms(), size=224, bs=bs
).normalize(imagenet_stats)

data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)
len(data.classes),data.c

## Train model with fastai

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)
learn.model

In [ ]:
learn.fit_one_cycle(1)

## Test trained model

In [ ]:
learn.predict(data.get(10))

## Save model with BentoML

In [ ]:
%%writefile pet_classification.py

from bentoml import BentoService, api, env, artifacts
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

@env(conda_pip_dependencies=['fastai'])
@artifacts([FastaiModelArtifact('pet_classifer')])
class PetClassification(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_classifer.predict(image)
        return str(result)

In [ ]:
from pet_classification import PetClassification

service = PetClassification.pack(pet_classifer=learn)

saved_path = service.save('/tmp/bentoml_archive')

print(saved_path)

# Deploy to Clipper cluster

In [ ]:
from clipper_admin import ClipperConnection, DockerContainerManager
from bentoml.deployment.clipper import deploy_bentoml

# Create clipper cluster connection
clipper_conn = ClipperConnection(DockerContainerManager())

# BentoML will deploy the BentoService archive as model to the clipper cluster
model_name, model_version = deploy_bentoml(clipper_conn, saved_path, api_name)

app_name = 'pet-classifier'

# Register a new application on the clipper cluster.  Application is an REST endpoint that expose
# to outside of the cluster.
app = clipper_conn.register_application(
    name=app_name,
    input_type='bytes',
    default_output='default result',
    slo_micros=100000
)

# Linking the deployed model with registered application on clipper cluster
linked = clipper_conn.link_model_to_app(app_name, model_name)